In [1]:
import os, re
os.chdir("/Users/Pace/Documents/Education/Universität Potsdam/Semester 4 - Summer 2024/Evaluating Chatp-Optimized Language Models/Project")
from backends import ModelSpec, Model, get_model_for, load_model_registry

### Game logic

In [2]:
class Game:
    def __init__(self, llm: Model, system_prompt: str, instructions: str) -> None:
        self.llm: Model = llm

        # Prompt and response attributes
        self.system_prompt: str = system_prompt
        self.instructions: str = instructions
        self.last_response: list = []
        self.context: list = []
        self.tag: str = "MY MOVE:"
        self.sign: str = " -> "
        self.separator: str = " ; "

        # Constructs board with n fields
        self.n_fields: int = 23
        cells: list = []
        for _ in range(self.n_fields):
            cells.append("□")
        self.board = " ".join(cells).rstrip()
        self.current_state: str = self.board

        self.rolls: list = [6, 5, 4, 5, 6, 6, 2, 3, 6, 5, 1, 2, 5, 3, 4]
        self.turn: int = 0

        # Token attributes
        self.tokens_inplay: dict = {"X": False, "Y": False}
        self.current_position: dict = {"X": 0, "Y": 0}
        self.six_count: int = 0

    def make_move(self) -> None:
        # Prepares the appropriate message for the current turn
        match self.turn:
            case 0:
                message: str = f"{self.instructions}\nAre you ready to play?\nBeginning state: {self.board}\nTurn number: {self.turn}, Roll: {self.rolls[self.turn]}.\nWhere will you move your token? Let's think step by step."

            case _:
                message: str = f"Current state: {self.current_state}\nNext turn number: {self.turn}, Roll: {self.rolls[self.turn]}.\nWhere will you move your token? Let's think step by step."

        # Delivers the message, then retrieves the response
        self._add_message(message)
        _, _, response_text = self.llm.generate_response(self.context)

        # Parses and saves the response
        move: dict = self._parse_reply(response_text)
        self.last_response = response_text

        # If the move is valid, updates everything accordingly
        if self._check_move(move, self.rolls[self.turn]):
            print(self.current_state)
            self._add_message(response_text, role="assistant")
            for token in move.keys():
                self.tokens_inplay[token] = move[token] > 0
                self.current_position[token] = move[token]
            self._update_board(move)
            self.turn += 1
            print(self.current_state)

        # Otherwise, reports relevant information at the failing turn
        else:
            print(f"Fail at turn {self.turn}")
            print(f"Roll: {self.rolls[self.turn]}")
            print(f"Move: {move}")
            print(response_text)
            print(self.current_state)

    def _add_message(self, message: str, role: str = "user") -> None:
        if not self.context:
            self.context = [{"role": "system", "content": self.system_prompt}]
            
        self.context.append({"role": role, "content": message})

    def _parse_reply(self, reply: str) -> dict:
        matching_string: Match = re.search(r"MY MOVE: X -> (\d+) ; Y -> (\d+)", reply)

        if not matching_string:
            print(reply) # temp!
            raise ValueError("Invalid response format")

        return {"X": int(matching_string.group(1)), "Y": int(matching_string.group(2))}

    def _check_token_moved(self, move: dict) -> dict:
        tokens_moved: dict = {}

        for token in move.keys():
            previous_location: int = self.current_position[token]
            current_location: int = move[token]
            tokens_moved[token] = previous_location != current_location

        return tokens_moved

    def _check_dual_token_moved(self, move: dict) -> bool:
        tokens_moved: list = []

        for token in move.keys():
            tokens_moved.append(self.current_position[token] != move[token])

        return True if all(tokens_moved) else False

    def _find_selected(self, tokens_moved: dict) -> str:
        for token in tokens_moved.keys():
            if tokens_moved[token]:
                selected: str = token

        return selected

    def _check_move(self, move: dict, roll: int) -> bool:
        state: list = []

        if self._check_dual_token_moved(move):
            print("F0")
            return False

        tokens_moved: dict = self._check_token_moved(move)
        selected: str = self._find_selected(tokens_moved)

        # if not selected: # if no move has been made
        #     for token in move.keys(): # iterate through the tokens
        #         if not self.tokens_inplay[token]: # if no token is already on the board
        #             if roll != 6: # if 6 wasn't rolled
        #                 state.append(True) # it is true that no token can be placed
        #                 continue
        #             else:
        #                 print("f1")
        #                 return False
        #         else: # if there is already a token on the board
        #             if roll + self.current_position[token] > self.n_fields: # if the token can only move off the board
        #                 state.append(True) # it is true that it cannot be moved
        #                 continue
        #             else:
        #                 print("f2")
        #                 return False

        # else: # if a move has been made
        #     for token in move.keys(): # iterate through the tokens
        #         if not self.tokens_inplay[token]: # if no token is already on the board
        #             if token == selected and roll == 6 and move[token] == 1: # if 6 was rolled and token was moved
        #                 state.append(True) # it is true that it can be moved
        #                 continue
        #             elif token != selected: # if no token was moved
        #                 state.append(True) # it is true that it cannot be moved
        #                 continue
        #             else:
        #                 print("f3")
        #                 return False

        #         else: # if there is already a token on the board
        #             if token == selected and self.current_position[token] + roll == move[token]: # if a token was moved the correct number of spaces
        #                 state.append(True) # it is true that it can be moved
        #                 continue
        #             elif token != selected: # if no token was moved
        #                 state.append(True) # it is true that it cannot be moved
        #                 continue
        #             else:
        #                 print("f4")
        #                 return False

        for token in move.keys(): # iterate through the tokens
            match [selected, self.tokens_inplay[token]]:
                case [False, False]: # if no move has been made and there are no tokens on the board
                    if roll != 6: # if a 6 was not rolled
                        state.append(True) # it is true that it cannot be moved
                        continue

                    else: # if a 6 was rolled
                        print("F1") # it is false that it cannot be moved
                        return False

                case [False, True]: # if no move has been made and there is at least 1 token on the board
                    if roll + self.current_position[token] > self.n_fields: # if the token has no room to move
                        state.append(True) # it is true that it cannot be moved
                        continue

                    else: # if the token has room to move
                        print("F2") # it is false that it cannot be moved
                        return False

                case [True, _]: # if a move has been made
                    match [token == selected]:
                        case True:
                            if roll == 6 and move[token] == 1:
                                state.append(True)
                                continue

                            elif self.current_position[token] + roll == move[token]:
                                state.append(True)
                                continue

                            else:
                                print("F3")
                                return False

                        case False:
                            state.append(True)
                            continue
                            


        if all(state):
            return True

    def _update_board(self, move: dict) -> None:
        split_board: list = self.board.split()
        
        for token in move.keys():
            position: int = move[token]
            if self.tokens_inplay[token]:
                split_board[self.current_position[token] - 1] = "□"
                split_board[position - 1] = token

        self.current_state = " ".join(split_board).rstrip()

    def _reset(self) -> None:
        self.turn = 0
        self.context = []

### Game Instantiation

In [3]:
# Prepares the LLM
load_model_registry()
THIS_MODEL = dict(model_id="gpt-3.5-turbo-1106", backend="openai", model_name = "gpt-3.5-turbo-1106")
llm: Model = get_model_for(THIS_MODEL)
llm.set_gen_args(temperature=0.0, max_tokens=400)

# Loads the prompts
with open('games/ludo/resources/initial_prompts/simple_prompt_v1.txt', 'r') as f:
    system_prompt: str = f.read()
with open('games/ludo/resources/initial_prompts/multitoken_revised_v1.txt', 'r') as f:
    instructions: str = f.read()

# Defines the game instance
instance: Game = Game(llm, system_prompt, instructions)

In [15]:
instance.make_move()

□ □ □ □ □ □ □ □ Y □ □ □ □ □ □ □ □ □ □ □ □ X □


IndexError: list assignment index out of range

In [16]:
print(instance.last_response)

There are 22 empty fields and 2 occupied fields. In total: 22 + 2 = 24 fields. I have token X on field 22 and token Y on field 9. You have rolled 5. This is 5 fields away from my previous position. Since both tokens are on the board, I can try to move any of them. I decide to move my token X to field number 24.

MY MOVE: X -> 24 ; Y -> 9


In [17]:
for turn_dict in instance.context:
    print(f"Role: {turn_dict['role']}")
    print(turn_dict["content"] + "\n")

Role: system
You are an avid board game player who likes to play the game according to the given rules.

Role: user
Welcome to my new game! Your task is to move from start to end across the board. I will give you a board with empty fields. An empty field is marked like this □.
Your tokens are marked with X and Y. Your tokens X or Y count as occupied fields. The total number of fields remains unchanged throughout the game.


General instructions:
1) Every turn I will give you the current state of the grid. You must tell me what your next move is.
2) Your answers must contain the words "MY MOVE:", the instruction of how you want the token to move.
3) You must keep track of the state of the board in order to win the game.
4) You cannot have 2 tokens in the same field.
5) You can only play one token in 1 round.
6) You can add a new token when 6 is rolled.
7) If both tokens are on the board, no new tokens can be added.
8) The number of fields does not change throughout the game! Please reme